# 📌 LLM-Based Fraud Report Generator

## 📝 Overview
This notebook leverages **OpenAI’s GPT-4** to generate structured fraud analysis reports for flagged credit card transactions. The model provides:
- ✅ **Fraud likelihood assessment** (🚨 Critical / ⚠️ High / 🔎 Moderate)
- ✅ **Key anomalies detected** in transaction patterns
- ✅ **Recommended actions** for fraud analysts to mitigate risk

Additionally, the project includes **Gradio integration** for **interactive report viewing**.

---

# 🔄 Workflow Overview
This fraud detection workflow follows a structured **6-step pipeline**:

### **1️⃣ Load and Preprocess Data**
- Load **credit card transaction data** (CSV file)
- Ensure **data validity and structure**
- Convert transactions into **dictionary format** for LLM processing

### **2️⃣ Initialize GPT-4 API**
- Set up **OpenAI API key securely**
- Configure **model parameters** (temperature, token limits)
- Define fraud risk assessment criteria

### **3️⃣ Generate Fraud Report using LLM**
- Pass transaction details to **GPT-4**
- Receive **AI-generated risk analysis**
- Categorize **fraud risk level** based on confidence scores

### **4️⃣ Save Reports for Future Reference**
- Store **fraud reports in a text file**  
- Ensure **auditability** for regulatory compliance

### **5️⃣ Create an Interactive UI (Gradio)**
- Enable **real-time fraud report viewing**
- Let users **query past reports** interactively

### **6️⃣ Deploy & Optimize**
- Deploy on **Hugging Face Spaces** or local web server
- Fine-tune **fraud detection models** for better accuracy

---




## **Step 1: Install & Import Dependencies** 
**This section installs the required libraries and imports necessary dependencies for fraud analysis and report generation.**


In [2]:
!pip install openai pandas numpy gradio

import openai
import numpy as np
import pandas as pd
import os
import gradio as gr


## **Step 2: Load Fraudulent Transactions**   
We load the transactions that were flagged as fraudulent from the model evaluation step.


In [3]:
import pandas as pd

# Define the path to the CSV file
fraud_cases_path = "/Users/adityaiyer/Desktop/Credit-Card-Fraud-Detection/outputs/detected_fraud_cases.csv"  # Ensure correct path

# Load fraud cases from CSV
fraud_cases = pd.read_csv(fraud_cases_path)

print(f"Fraudulent transactions successfully loaded from: {fraud_cases_path}")
print(fraud_cases.head())  # Display first few rows


Fraudulent transactions successfully loaded from: /Users/adityaiyer/Desktop/Credit-Card-Fraud-Detection/outputs/detected_fraud_cases.csv
         V1         V2        V3         V4        V5         V6        V7  \
0 -1.128572  -7.731412  4.496687 -10.772688  6.498647  -9.681829 -2.209807   
1  1.074242   0.463557  1.736440  -3.719073  4.247607   0.155738 -1.790334   
2  0.109825 -10.029916  8.962974 -15.017672  8.561871 -10.711361  0.628847   
3 -1.134173  -2.713928  2.526858  -5.619427  5.897613  -4.111121 -2.273396   
4 -1.087383  -1.718806  1.478931  -2.470834  0.196116  -1.657888 -1.464989   

          V8        V9       V10  ...       V21        V22       V23  \
0 -12.853497  8.385738 -7.200934  ... -2.202172   4.259077  0.468727   
1  -1.446658  0.283518 -2.569872  ...  0.318113   0.550153 -0.547934   
2 -22.272181 -9.916865 -8.166069  ...  5.001409 -11.139194  4.562954   
3  -8.737723  2.834497 -5.620241  ...  1.501860   3.154640  1.185810   
4  -2.408722 -1.391549  0.167467  

## **Step 3: Configure API Access**   
This step establishes a **secure connection** to OpenAI’s API, enabling GPT-4 to generate fraud reports. API credentials must be **securely managed** to prevent **unauthorized access** and potential 


In [4]:
# 2. Set your OpenAI API Key
client = openai.OpenAI(api_key="Enter your API Key here")





## **Step 4: Generate Fraud Reports using GPT-4**   
We send flagged transactions to **GPT-4**, which generates **detailed structured fraud reports** for each case.  
These reports provide:
- **Risk assessment** (🚨 Critical / ⚠️ High / 🔎 Moderate)
- **Key anomalies detected** in transaction behavior
- **Recommended actions** for fraud analysts to mitigate risk  

Each report is stored and can be accessed later for further investigation.


In [20]:
def generate_fraud_report(transaction_details, model_confidence, action_taken):
    """
    Uses OpenAI GPT-4 to generate a structured fraud report with improved readability.
    """
    # Add Fraud Probability Interpretation
    fraud_risk = (
        "Critical Risk 🚨" if model_confidence >= 96 else 
        "High Risk ⚠️" if model_confidence >= 90 else 
        "Moderate Risk 🔎"
    )
    prompt = f"""
    You are a financial fraud analyst. A credit card fraud detection system has flagged the following transaction as fraudulent.

    ---
    **🔍 Transaction Details:**  
    {transaction_details}  

    **📊 Model Confidence:** {model_confidence}%  
    **🚨 Action Taken:** {action_taken}  

    ---
    ### **1️⃣ Why This Transaction Was Flagged**
    - Identify unusual spending patterns based on PCA-transformed variables (V1-V28).
    - Explain what these variables might represent in real-world scenarios (e.g., sudden location change, high spending in short periods, unusual merchant category).
    - Compare the flagged transaction with past fraudulent behaviors in the dataset.

    ---
    ### **2️⃣ Key Insights from the Analysis**
    - Clearly explain what anomalies were found.
    - If possible, relate the PCA-transformed values to real-world factors (e.g., transaction location, spending frequency).
    - Provide a simple risk assessment: *High Risk / Moderate Risk / Low Risk*.

    ---
    ### **3️⃣ Recommended Actions**
    - **Urgent:** Immediate steps to verify or prevent fraud (e.g., contacting the customer, temporarily freezing the account).
    - **Investigation:** How fraud analysts can further verify the case.
    - **Prevention:** Suggested AI system improvements to prevent similar fraud cases.

    ---
    **⚠️ Important:** This is an AI-generated fraud report. It should be reviewed and confirmed by human fraud analysts before taking final action.
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a financial fraud analyst."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


## **Step 5: Generate Reports for Each Fraud Case**   
We iterate through the **flagged fraud transactions**, passing them to **GPT-4** for analysis.  
For each transaction, the model generates a **structured fraud investigation report** detailing:
- **Unusual transaction patterns**
- **Model confidence score & fraud risk level**
- **Recommended next steps for fraud analysts**  

The generated reports are then **stored for future reference and auditing**.


In [21]:
# Generate Fraud Reports for Each Fraud Case
fraud_reports = []

for i in range(min(3, len(fraud_cases))):  # Limiting to 3 reports for testing
    transaction_details = fraud_cases.iloc[i].to_dict()  # Convert row to dictionary
    model_confidence = np.random.randint(85, 99)  # Simulated confidence score
    action_taken = "Account Suspended"

    # Generate Report using LLM
    report = generate_fraud_report(transaction_details, model_confidence, action_taken)
    
    # Store Report
    fraud_reports.append(report)

# Display Generated Reports
for idx, report in enumerate(fraud_reports, 1):
    print("\n" + "="*50)  # 🔹 Add a separator for clarity
    print(f"📄 **Fraud Report {idx}:**")
    print("="*50)
    print(report)
    print("\n" + "="*50)  # 🔹 End of report separator



📄 **Fraud Report 1:**
### **1️⃣ Why This Transaction Was Flagged**
This transaction was flagged due to several significant anomalies compared to normal spending behaviors based on the PCA-transformed variables. These variables, which have been anonymized (labeled as V1-V28) for privacy reasons, can be associated with real-world transaction behaviors such as transaction location, spending frequency, and merchant category.

- In this case, we observe unusually high/low values for variables like V2, V4, V5, V6, V8, V10, V11, V13, V15, V17, V18, V19, and V28.
- For instance, the highly negative values of V4, V6, V8 and V10 might indicate highly unusual merchant categories or spending patterns that are significantly different from the norm.
- Similarly, high values for V5, and V9 suggest sudden substantial charges that are atypical for this account which could suggest a suspicious activity or a possible case of fraud.

Furthermore, comparing these values with past fraudulent behaviors in t

## **Step 6: Save Reports for Future Reference**  
We store fraud reports in a text file for **later review, auditing, or reporting**.  
This ensures that flagged transactions can be analyzed later for **regulatory compliance** and **fraud prevention improvements**.


In [22]:
# Save Fraud Reports to a Text File
output_path = "/Users/adityaiyer/Desktop/Credit-Card-Fraud-Detection/outputs/fraud_reports.txt"

with open(output_path, "w") as f:
    for idx, report in enumerate(fraud_reports, 1):
        f.write(f"\n📄 Fraud Report {idx}:\n{report}\n")

print(f"\n All Fraud Reports Successfully Saved to {output_path}")



 All Fraud Reports Successfully Saved to /Users/adityaiyer/Desktop/Credit-Card-Fraud-Detection/outputs/fraud_reports.txt


### **Step 7: Interactive UI with Gradio**  
We build a **Gradio-based interactive tool** that allows users to **query fraud reports**.  
This enables fraud analysts or stakeholders to **view AI-generated reports** for specific transactions in real time.


In [23]:
#  Define a function to display fraud reports interactively
def display_fraud_report(transaction_index):
    if transaction_index < 1 or transaction_index > len(fraud_reports):
        return "❌ Invalid transaction number. Please enter a number between 1 and " + str(len(fraud_reports))
    return fraud_reports[transaction_index - 1]  # Adjusting for 0-based index

#  Create a Gradio Interface
interface = gr.Interface(
    fn=display_fraud_report,
    inputs=gr.Number(label="Enter Transaction Number (1 to {})".format(len(fraud_reports))),
    outputs="text",
    title="📄 Credit Card Fraud Report Viewer",
    description="Enter a transaction number to view its fraud report."
)

# Launch the Web Interface
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://ac8ef78f5e823ef165.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
